# Paths in Graphs

## Initial Foray

Write initials on top of each other, count dots, regions, segments, then add dots and regions and subtract the number of segments.

I got one.

This is a planar graph, and we invoked Euler's formula: `n - m + r = 2`, but here we didn't count the giant region outside.

## Properties of Social Networks

Just graphs, but real-world networks. They have some distinctive properties (Watts and others):

* "small worlds" (short paths) (Stan Milgram experiment - 6 degrees of separation)
* "cliqueish" (high clustering coefficient)

Graphs are "small world" if nodes have relatively small degree, but a relatively short path to other arbitrary nodes. 

Consider

               clique      ring        balanced tree    hypercube
               deg path    deg path    deg path         deg path
    
    Theta(1)       *       *           *
    Theta(logn)                            *            *   *
    Theta(n)   *               *                        

## Clustering Coefficient

`CC(V):` trying to capture "cliqueishness." `v` a node, `K_v` is its degree, `N_v` is the number of links between neighbors of `v`

<img src="Screen_Shot_2015-12-03_at_6.14.34_PM.png">

We compute:

\begin{equation}
CC(V) = \frac{2 N_v}{K_v (K_v - 1)}
\end{equation}

Here, `(2*1)/(4*3) = 1/6`.

This represents the _fraction of possible interconnections_. $0 \leq CC(V) \leq 1$. If $CC(V) = 0$, we have a star graph, and if $CC(V) = 1$, we have a clique graph.

Note that $K_v (K_v - 1)/2$ is the maximum number of pairs we can form between $K_v$ items.

The clustering coefficient for a _graph_ is the _average_ of the clustering coefficent of all the nodes.

### Quiz

<img src="Screen_Shot_2015-12-03_at_6.21.51_PM.png">

$K_v = 4$

$N_v = 2$

$2(N_v)/(K_v (K_v - 1)) = 4/12 = 1/3$

$\Rightarrow CC(ORD) = 1/3$

In [1]:
def make_link(G, node1, node2):
    if node1 not in G:
        G[node1] = {}
    (G[node1])[node2] = 1
    if node2 not in G:
        G[node2] = {}
    (G[node2])[node1] = 1
    return G

flights = [('ORD', 'SEA'), ('ORD', 'LAX'), ('ORD', 'DFW'), ('ORD', 'PIT'),
           ('SEA', 'LAX'), ('LAX', 'DFW'), ('ATL', 'PIT'), ('ATL', 'RDU'),
           ('RDU', 'PHL'), ('PIT', 'PHL'), ('PHL', 'PVD')]

G = {}
for (x, y) in flights:
    make_link(G, x, y)

print G    
    
def clustering_coefficient(G, v):
    neighbors = G[v].keys()
    if len(neighbors) == 1: 
        return -1.0
    links = 0
    for w in neighbors:
        for u in neighbors:
            if u in G[w]: 
                links += 0.5
    return 2.0 * links / (len(neighbors) * (len(neighbors) - 1))

# O'Hare's clusterng coefficient
print clustering_coefficient(G, "ORD")

# The average coefficient (the score for the graph)
total = 0
for v in G.keys():
    total += clustering_coefficient(G,v)

print total / len(G)

{'RDU': {'ATL': 1, 'PHL': 1}, 'PHL': {'RDU': 1, 'PIT': 1, 'PVD': 1}, 'ATL': {'RDU': 1, 'PIT': 1}, 'LAX': {'ORD': 1, 'DFW': 1, 'SEA': 1}, 'PVD': {'PHL': 1}, 'SEA': {'ORD': 1, 'LAX': 1}, 'ORD': {'PIT': 1, 'DFW': 1, 'LAX': 1, 'SEA': 1}, 'PIT': {'ORD': 1, 'ATL': 1, 'PHL': 1}, 'DFW': {'ORD': 1, 'LAX': 1}}
0.333333333333
0.222222222222


Social networks often have clustering coefficients of ~0.1.

## Connected Components

* Is everything connected?
* Are there isolated sets of nodes?
* Can all the nodes communicate?

We count "disconnected components" as clusters of nodes that can all communicate amongst themselves. So, a graph where all the nodes can communicate has "1 disconnected component" (not zero). If there are two clusters, then we say there are "2 disconnected components," etc.

In [2]:
connections = [('a', 'g'), ('a', 'd'), ('d', 'g'), ('g', 'c'),
               ('b', 'f'), ('f', 'e'), ('e', 'h')]
G = {}
for (x, y) in connections:
    make_link(G, x, y)

In [3]:
def mark_component(G, node, marked):
    """
    G is a graph
    node is a node (here keyed by a letter)
    marked is a dictionary tracking which nodes are marked
    """
    marked[node] = True                                           # once per node
    total_marked = 1                                              # once per node
    for neighbor in G[node]:
        if neighbor not in marked:
            total_marked += mark_component(G, neighbor, marked)   # once per edge in the graph
    return total_marked                                           # once per node

In [4]:
def list_component_sizes(G):
    """
    G is a graph
    
    print the number of connected sub-graphs and the number of nodes in each various
    sub-graph
    """
    # make a dictionary to track which nodes we've visited
    marked = {}
    # loop through all the nodes in the graph
    for node in G.keys():
        # if we haven't visited the node...
        if node not in marked:
            # then mark all the nodes connected to it and count the total number of
            # nodes in the sub-graph
            print "Component containing", node, ": ", mark_component(G, node, marked)

In [5]:
list_component_sizes(G)

Component containing a :  4
Component containing b :  4


## Running Time of Connected Component Counting

Two principle flavors:

* Depth first search (what we have above - $\Theta(n + m)$)
* Breadth first search

`n` nodes and `m` edges.

## Checking Pairwise Connectivity

In [6]:
##################################################################
# Traversal...
# Call this routine on nodes being visited for the first time
def mark_component(G, node, marked):
    marked[node] = True
    total_marked = 1
    for neighbor in G[node]:
        if neighbor not in marked:
            total_marked += mark_component(G, neighbor, marked)
    return total_marked

def check_connection(G, v1, v2):
    marked = {}
    n_marked = mark_component(G, v1, marked)
    return v2 in marked
    
def make_link(G, node1, node2):
    if node1 not in G:
        G[node1] = {}
    (G[node1])[node2] = 1
    if node2 not in G:
        G[node2] = {}
    (G[node2])[node1] = 1
    return G

In [7]:
def test():
    edges = [('a', 'g'), ('a', 'd'), ('g', 'c'), ('g', 'd'), 
             ('b', 'f'), ('f', 'e'), ('e', 'h')]
    G = {}
    for v1, v2 in edges:
        make_link(G, v1, v2)
        
    assert check_connection(G, "a", "c") == True
    assert check_connection(G, 'a', 'b') == False

In [8]:
test()

In [9]:
check_connection(G, "a", "c")

True

In [10]:
check_connection(G, "a", "b")

False

In [11]:
check_connection(G, "a", "d")

True

In [12]:
check_connection(G, "g", "d")

True

In [13]:
check_connection(G, "g", "f")

False

## Pairwise Shortest Path

"The Kevin Bacon Game"

Given `G` and `v1` (Kevin Bacon) and `v2` (Emma Watson), find the shortest path between them.

## Depth vs. Breadth First Searches

Depth first searches, well, go deep first. Breadth first searches exhaust the neighborhood before going very far.

## Depth First without recursion

Open list: "Todo list"

* Grab _last_ element of open list. (Start with some node at the beginning.)
* Mark any unmarked neighbors and add them to open list at the end (and then remove that node from the list).
* Repeat until nothing open.

As an example:

* Consider: `a-b-c-d-e-f-g`
* Start with `d` => To-do list is `ce`
* Visit `e`, then add un-marked neightbors to the end of the list.
    * `a-b-c-d1-e2-f-g` and `cf`
* Pull from the end of the todo list, `f`, visit unmarked neighbors, etc.
    * `a-b-c-d1-e2-f3-g` and `cg`
* Pull from the end of the todo list, `g`, visit unmarked neighbors, etc.
    * `a-b-c-d1-e2-f3-g4` and `c`
* Pull from the end of the todo list, `c`, visit unmarked neighbors, etc.
    * `a-b-c5-d1-e2-f3-g4` and `b`
* Pull from the end of the todo list, `b`, visit unmarked neighbors, etc.
    * `a-b6-c5-d1-e2-f3-g4` and `a`
* Pull from the end of the todo list, `a`, visit unmarked neighbors, etc.
    * `a7-b6-c5-d1-e2-f3-g4` and ``
* Done.

## Breadth First without recursion

Open list: "Todo list"

* Grab _first_ element of open list. (Start with some node at the beginning.)
* Mark any unmarked neighbors and add them to open list at the end (and then remove that node from the list).
* Repeat until nothing open.

As an example:

* Consider `a-b-c-d-e-f-g`
* Start with `d`, todo is `ce`
    * `a-b-c-d1-e-f-g` and `ce`
* Pull from the front of the list, but add open neighbors to the end
    * `a-b-c2-d1-e-f-g` and `eb`
* Pull from the front of the list, `e`, but add open neighbors to the end
    * `a-b-c2-d1-e3-f-g` and `bf`
* Pull from the front of the list, `b`, but add open neighbors to the end
    * `a-b4-c2-d1-e3-f-g` and `fa`
* Pull from the front of the list, `f`, but add open neighbors to the end
    * `a-b4-c2-d1-e3-f5-g` and `ag`
* Pull from the front of the list, `a`, but add open neighbors to the end
    * `a6-b4-c2-d1-e3-f5-g` and `g`
* Pull from the front of the list, `g`, but add open neighbors to the end
    * `a6-b4-c2-d1-e3-f5-g7` and ``
* Done.

## Searcing a Tree

## Marvel "Social" Network

<img src="Screen_Shot_2015-12-14_at_6.08.47_PM.png">

In [14]:
cat marvel.py

#!/usr/bin/env python

import csv
# import time


def make_link(G, node1, node2):
    if node1 not in G:
        G[node1] = {}
    (G[node1])[node2] = 1
    if node2 not in G:
        G[node2] = {}
    (G[node2])[node1] = 1
    return G


def read_graph(filename):
    """
    read an undirected graph in csv format. each line is an edge
    """
    tsv = csv.reader(open(filename), delimiter='\t')
    G = {}
    for (node1, node2) in tsv:
        make_link(G, node1, node2)
    return G

marvelG = read_graph('uniq_edges.tsv')


def path(G, v1, v2):
    distance_from_start = {}
    path_from_start = {}
    open_list = [v1]
    distance_from_start[v1] = 0
    path_from_start[v1] = [v1]
    while len(open_list) > 0:
        current = open_list[0]
        del open_list[0]
        for neighbor in G[current].keys():
            if neighbor not in distance_from_start:
                distance_from_start[neighbor] = \
                    distance_from_start[current] + 1
                path_from_

## Single Source Shortest Paths

How central is `v1`? One definition is the _average shortest path length to all the other nodes_.

The shortest path run lenght is $\Theta(n + m)$. So, a naive repetition for all $n$ nodes is $\Theta(n(n+m))$.

## All targets in one shot

How can we find all distances from `v1` to the rest of the graph faster? Faster than $\Theta(n^2 + nm)$?

* We can't. Deal with it?
* Use a smaller graph?
* Search for something that isn't there? - THIS IS IT
* Do the searches backwards?

## Centrality

Modify the path-finding algorithm to find the _average_ of the shortest path distance to each (reachable) node - find the **centrality** measure for $v_1$. Note that this notion of centrality only makes sense for a set of connected nodes. If the graph is disconnected, it can return odd answers.

In [16]:
import csv

def make_link(G, node1, node2):
    if node1 not in G:
        G[node1] = {}
    (G[node1])[node2] = 1
    if node2 not in G:
        G[node2] = {}
    (G[node2])[node1] = 1
    return G


def read_graph(filename):
    """
    read an undirected graph in csv format. each line is an edge
    """
    tsv = csv.reader(open(filename), delimiter=',')
    G = {}
    for (node1, node2) in tsv:
        make_link(G, node1, node2)
    return G

marvelG = read_graph('source.csv')

In [17]:
def path(G, v1, v2):
    distance_from_start = {}
    path_from_start = {}
    open_list = [v1]
    distance_from_start[v1] = 0
    path_from_start[v1] = [v1]
    while len(open_list) > 0:
        current = open_list[0]
        del open_list[0]
        for neighbor in G[current].keys():
            if neighbor not in distance_from_start:
                distance_from_start[neighbor] = \
                    distance_from_start[current] + 1
                path_from_start[neighbor] = \
                    path_from_start[current] + [neighbor]
                if neighbor == v2:
                    return distance_from_start[v2]
                open_list.append(neighbor)
    return False

In [18]:
def centrality(G, v):
    distance_from_start = {}
    path_from_start = {}
    open_list = [v]
    distance_from_start[v] = 0
    while len(open_list) > 0:
        current = open_list[0]
        del open_list[0]
        for neighbor in G[current].keys():
            if neighbor not in distance_from_start:
                distance_from_start[neighbor] = \
                    distance_from_start[current] + 1
                path_from_start[neighbor] = \
                    path_from_start[current] + [neighbor]
                open_list.append(neighbor)
    return sum(distance_from_start.values() + 0.0)/len(distance_from_start)

## Bridge Edges

<img src="Screen_Shot_2016-01-06_at_6.05.57_PM.png">

Tarjan's rooted tree (note the "non-tree edges"):

<img src="Screen_Shot_2016-01-06_at_6.09.58_PM.png">

1. write graph as tree
2. "post order" the nodes

"Post ordering" works by numbering nodes, but we must always number all the children to the left and right of a node before we may number a given node.

After post-ordering:

<img src="Screen_Shot_2016-01-06_at_6.14.15_PM.png">

## Post-order a tree

Got "6"